In [ ]:
import json
import os
import torch
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import scipy
from DataLoader import EuroSAT
from torchvision import datasets , transforms
from torchvision.transforms import ToTensor


In [ ]:
# Hyperparameters
transform = transforms.Compose([
    ToTensor() , 
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) ,    
])
BATCH_SIZE = 8 
LR = 0.001
device = 'cuda' if torch.cuda.is_available() else "cpu"
device


Dataset Prep

In [ ]:
train_csv = pd.read_csv("EuroSAT/train.csv" , index_col = 0)
test_csv = pd.read_csv("EuroSAT/test.csv" , index_col = 0)
val_csv = pd.read_csv("EuroSAT/validation.csv" , index_col = 0)

train_csv = train_csv.sort_values(axis = 0 , by = ['ClassName'])
test_csv = test_csv.sort_values(axis = 0 , by = ['ClassName'])
val_csv = val_csv.sort_values(axis = 0 , by = ['ClassName'])

In [ ]:
with open("EuroSAT/label_map.json" , 'r') as file:
    labels = json.load(file)
    class_names = list(labels.keys())
class_names 

In [ ]:
train_set = []
test_set = []
val_set = []
sets = [train_csv , val_csv , test_csv]

for i , set in enumerate(sets):
    if i == 0:
        for index, row in set.iterrows():
            train_set.append(list(row))
    elif i == 2:
        for index, row in set.iterrows():
            test_set.append(list(row))
            
    else: 
        for index, row in set.iterrows():
            val_set.append(list(row))           

In [ ]:
train = EuroSAT(parent_dir = "EuroSAT" , data = train_set , transform = transform)
val = EuroSAT(parent_dir = "EuroSAT" , data = val_set , transform = transform)
test = EuroSAT(parent_dir = "EuroSAT" , data = test_set , transform = transform)

len(train) , len(val) , len(test)

In [ ]:
train_loader = torch.utils.data.DataLoader(train , shuffle = True , batch_size=BATCH_SIZE)
val_loader = torch.utils.data.DataLoader(val , shuffle = True , batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test, shuffle = False , batch_size=BATCH_SIZE)

len(train_loader) , len(val_loader)  , len(test_loader)

Visualize some samples

In [ ]:
train_iter = iter(train_loader)
first_batch = next(train_iter)
images , labels  = first_batch

images.shape , labels.shape

In [ ]:
formatted_images  = []
for image in images:
    image = image.permute(1 , 2 , 0)
    image = image.numpy()
    formatted_images.append(image)
    
images[0].shape , formatted_images[0].shape

In [ ]:
nrows = 2
ncolumns = 4
fig, axs = plt.subplots(nrows, ncolumns, figsize=(15, 6))

# Flatten the axs array to simplify accessing individual subplots
axs = axs.flatten()

for i in range(len(images)):
    ax = axs[i]  # Access the individual subplot
    ax.imshow(formatted_images[i])  # Display the image
    ax.set_title(class_names[labels[i]])  # Set the title to the class name of the image
    ax.axis('off')  # Hide the axis

plt.show()


NN From Scratch

In [ ]:
class NeuralNetwork:
    def __init__(self , input_dim , hidden_dim , output_dim , loss_function = 'mse'):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim 
        self.output_size = output_dim
        self.loss_func = loss_function
        
        self.w1 = np.random.randn(self.input_dim , self.hidden_dim)
        self.b1 = np.zeros((1 , self.hidden_dim))
        self.w2 = np.random.randn(self.hidden_dim , self.output_dim)
        self.b2 = np.zeros((1 , self.output_dim))
        
    def feedforward(self , x): 
        self.z1 = np.dot(x , self.w1) + self.b1
        self.a1 = torch.sigmoid(self.z1)
        self.z2 = np.dot(self.z2 , self.w2) + self.b2
        
        if self.loss_func == "binary_cross_entropy":
            self.a2 = torch.softmax(self.z2)
            
        else : 
            self.a2 = torch.sigmoid(self.z2)
            
        return self.a2  
    
    def backward(self , X , y , learning_rate):
        
        m = X.shape[0]
        
        if self.loss_func == "mse" or self.loss_func == "binary_cross_entropy":
            self.dz2 = self.a2 - y # aka loss
            
        elif self.loss_func == "log_loss" : 
            self.dz2 = -(y/self.a2 - (1-y)/(1-self.a2))
            
        else:
            raise ValueError("Invalid Loss Function")
    
        # gradients    
        self.dw2 = (1 / m) * np.dot(self.a1.T, self.dz2)
        self.db2 = (1 / m) * np.sum(self.dz2, axis=0, keepdims=True)
        self.dz1 = np.dot(self.dz2, self.weights2.T) * self.sigmoid_derivative(self.a1)
        self.dw1 = (1 / m) * np.dot(X.T, self.dz1)
        self.db1 = (1 / m) * np.sum(self.dz1, axis=0, keepdims=True)
        
        # Update
        self.weights2 -= learning_rate * self.dw2
        self.bias2 -= learning_rate * self.db2
        self.weights1 -= learning_rate * self.dw1
        self.bias1 -= learning_rate * self.db1     

NN using Pytorch library